# Ejercicio FASTAPI API
Para este ejercicio tendrás que desplegar un modelo de machine learning en una API para su consumo. Ya tienes un modelo entrenado,desarrollarás una API que permita consumir dicho modelo desde cualquier otra tecnología.

**Se presenta el siguiente caso de uso**

Una empresa distribuidora de ámbito nacional pretende utilizar un modelo desarrollado por el departamento de data science, con el que consiguen una predicción de las ventas a partir de los gastos en marketing de anuncios en televisión, radio y periódicos. Quieren incorporar estos datos dentro de su página web interna, donde comparten todo tipo de información relativa a resultados de la empresa, ventas, adquisiciones, etc... La web está desarrollada en AngularJS, mientras que el modelo se desarrolló en Python, por lo que precisamos de una interfaz de comunicación entre ambos sistemas.

El equipo de desarrollo necesita que implementes un microservicio para que ellos puedan consumir el modelo desde la propia web, comunicándose con una BBDD para ingestar o reentrenar el modelo. No vale base de datos en csv. El microservicio tiene que cumplir las siguientes características:
1. Ofrezca la predicción de ventas a partir de todos los valores de gastos en publicidad. (/predict)
2. Un endpoint para almacenar nuevos registros en la base de datos que deberás crear previamente.(/ingest)
3. Posibilidad de reentrenar de nuevo el modelo con los posibles nuevos registros que se recojan. (/retrain)


**NOTAS**: 
1. Deberás desplegarlo desde un repositorio de github.
2. Ojo con la ruta para hacer el load de tu modelo y datos, comprueba cual es la ruta en la que está buscándolo.
3. El desarrollo de un modelo de machine learning no es el objetivo del ejercicio, sino el desarrollo de una API con FastAPI.
4. Deberán pasar los tests para verificar que funcionan correctamente con pytest test_api.py desde terminal.

**Entregable**: repositorio github.

In [9]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
from sqlalchemy import create_engine, Column, Integer, Float, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import joblib
import pandas as pd
from sklearn.linear_model import LinearRegression

app = FastAPI()

# Configurar la base de datos
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Definir el modelo de datos
class Advertising(Base):
    __tablename__ = "advertising"
    id = Column(Integer, primary_key=True, index=True)
    TV = Column(Float)
    Radio = Column(Float)
    Newspaper = Column(Float)
    Sales = Column(Float)

Base.metadata.create_all(bind=engine)

# Cargar el modelo
model = joblib.load('data/advertising_model.pkl')

# Pydantic models
class AdData(BaseModel):
    TV: float
    Radio: float
    Newspaper: float

class AdDataIngest(BaseModel):
    TV: float
    Radio: float
    Newspaper: float
    Sales: float

@app.post("/predict")
def predict(data: List[AdData]):
    df = pd.DataFrame([d.dict() for d in data])
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}

@app.post("/ingest")
def ingest(data: AdDataIngest):
    db = SessionLocal()
    ad_record = Advertising(TV=data.TV, Radio=data.Radio, Newspaper=data.Newspaper, Sales=data.Sales)
    db.add(ad_record)
    db.commit()
    db.refresh(ad_record)
    return {"message": "Data ingested successfully"}

@app.post("/retrain")
def retrain():
    db = SessionLocal()
    ads = db.query(Advertising).all()
    df = pd.DataFrame([{"TV": ad.TV, "Radio": ad.Radio, "Newspaper": ad.Newspaper, "Sales": ad.Sales} for ad in ads])
    X = df[["TV", "Radio", "Newspaper"]]
    y = df["Sales"]
    new_model = LinearRegression()
    new_model.fit(X, y)
    joblib.dump(new_model, 'data/advertising_model.pkl')
    return {"message": "Model retrained successfully"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

/var/folders/mq/mnz2dh_90hx7l6l083kjws6w0000gn/T/ipykernel_47162/3621695774.py:17: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


RuntimeError: asyncio.run() cannot be called from a running event loop